In [8]:
#importing libraries
import numpy as np
import pandas as pd
#keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
import matplotlib.pyplot as plt
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split 
from keras.wrappers.scikit_learn import KerasClassifier




from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Input, Dropout
from keras import Sequential

# for combining all the hyper-parameters
import itertools 
from time import time



In [2]:
# Grid search for the best parameter
g_eta = [0.1,0.2,0.3]
g_batchSize = [10,20,25,30]
g_hiddeLayerunit1 = [4,8,10]
g_momentum = [0.6,0.8,0.9]
g_afHiddenLayerunit1 = ["relu", "tanh"]


In [3]:
# reading training and testing data from the csv file
trainDf = pd.read_csv ('data/csv/monks-3Train.csv',header=None).to_numpy()
testDf = pd.read_csv ('data/csv/monks-3Test.csv',header=None).to_numpy()
trainDf = minmax_scale(trainDf, feature_range=(0,1), axis=0)
testDf = minmax_scale(trainDf, feature_range=(0,1), axis=0)

In [4]:
#Preparing trainDf for training
trainX = trainDf[:, 1:7]
trainY = trainDf[:, 0]
#trainX = scaler.fit_transform(trainX)
#trainY = scaler.fit_transform(trainY)

In [5]:
#Preparing testDf for validation
testX = testDf[:, 1:7]
testY = testDf[:, 0]


In [6]:
# All combination of the hyper-paramers
gridParameters=[g_eta,g_batchSize,g_hiddeLayerunit1,g_momentum,g_afHiddenLayerunit1]
allcombination = list(itertools.product(*gridParameters))


From the grid search I was able to get the best hyperparameters
eta = 0.1
batch size = 10
hiddenLayer = 10
momentum = 0.7
hiddenLayer af = relu

(0.1, 10, 10, 0.9, 'relu') - Then added a new hidden layer with activation unit of tanh and get accuracy of 99%

In [ ]:
def create_model(lr=0.1,momentum=0.9,g_hiddeLayerunit1=4,g_hiddeLayerunit2=4,activation1="tanh",activation2="tanh",activation3="sigmoid",init_mode='uniform'):
    model = Sequential()
    model.add(Dense(g_hiddeLayerunit1, input_dim=6, kernel_initializer=init_mode, activation=activation1))
    model.add(Dense(g_hiddeLayerunit2, kernel_initializer=init_mode, activation=activation2))
    model.add(Dense(1, activation3))
    sgd = SGD(lr=lr, momentum=momentum, nesterov=False)
    model.compile(optimizer=sgd, loss='mean_squared_error',metrics=['accuracy'])
    return model

start=time()
model = KerasClassifier(build_fn=create_model)

# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
g_hiddeLayerunit1=[4,8,16]
g_hiddeLayerunit2=[4,8,16]
activation1 = ["relu", "tanh","sigmoid"]
activation2 = ["relu", "tanh","sigmoid"]
activation3 = ["relu", "tanh","sigmoid"]
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
lr=np.arange(0.1, 0.9, 0.1).tolist()
momentum=np.arange(0.1, 0.9, 0.1).tolist()
param_grid = dict(
    lr=lr,
    g_hiddeLayerunit1=g_hiddeLayerunit1,
    g_hiddeLayerunit2=g_hiddeLayerunit2,
    activation1=activation1,
    activation2=activation2,
    activation3=activation3,
    batch_size=batch_size,
    init_mode=init_mode,
    epochs=epochs)


grid = GridSearchCV(estimator=model, param_grid=param_grid,n_jobs=2,cv=3)
grid_result = grid.fit(trainX, trainY)

end=time()

print("Total Running Time: %f",end-start)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
print("Mean\tSTD\tParams")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
